In [1]:
from scale import process_to_size
import os

In [2]:
def get_file_paths(directory):
   """goes through the given directory and returns a list of the paths to all the files"""
   file_paths = []
   
   # walk through directory and subdirectories
   for root, dirs, files in os.walk(directory):
       for file in files:
           # create full file path
           file_path = os.path.join(root, file)
           file_paths.append(file_path)
           
   return file_paths

directory = "./selected/"
paths = get_file_paths(directory)

./selected/nm0705356_rm860471040_1989-7-23_2007.jpg
./selected/nm0002122_rm3114050048_1951-10-30_2011.jpg
./selected/nm0000553_rm2186975488_1952-6-7_2002.jpg
./selected/nm0004812_rm1021102336_1951-7-24_1975.jpg
./selected/nm0911320_rm2216791808_1964-11-14_2007.jpg
./selected/nm0001719_rm93310464_1934-2-13_1972.jpg
./selected/nm2832695_rm830128640_1985-2-7_2008.jpg
./selected/nm0000330_rm21662976_1944-2-13_2010.jpg
./selected/nm0917060_rm1072422144_1978-9-3_2011.jpg
./selected/nm0035488_rm2776335616_1966-12-4_2011.jpg


In [3]:
from tqdm import tqdm
from PIL import Image

# This is the scaling part

for path in tqdm(paths):
    # print(path)
    img = process_to_size(path,target_size=224)
    img.save(f"./scaled/{path[11:]}") # the slicing is to remove the "selected" directory from the path

100%|██████████| 102750/102750 [14:43<00:00, 116.29it/s]


In [8]:
#This script is for visualizing how the scaling affected images

new_paths = get_file_paths("./scaled/")
for i in tqdm(zip(new_paths,paths)):
    scaled = Image.open(i[0])
    orig = Image.open(i[1])
    if (scaled.size[0] != 224) or (scaled.size[1] != 224):
        display(orig),display(scaled)
        print(scaled.size,orig.size)

102750it [00:59, 1732.07it/s]


In [ ]:
from pathfxn import get_file_paths

# The following three blocks are largely for creating Training and Testing sets in .json files

df_dim = pd.read_csv("imdb_crop_dimensions.csv")
scaled = get_file_paths("./scaled/")
base = []
training = []
for elem in scaled:
    base += [elem[9:]]
df_dim['Clean_Path'] = df_dim.apply(lambda x: x["Path"][15:],axis=1)
base_df = df_dim[df_dim["Clean_Path"].isin(base)].copy()
base_df.apply(lambda x: training.append((x["Name"],x["Clean_Path"])),axis=1)
names = base_df["Name"].unique()
other_imgs = df_dim[(df_dim["Name"].isin(names))&(~df_dim["Clean_Path"].isin(base))]
testing = []
for name in names:
    person = df_dim[df_dim["Name"]==name]
    if len(person) < 10:
        sampled = person.sample(len(person))
    else:
        sampled = person.sample(10)
    sampled.apply(lambda x: testing.append((x["Name"],x["Path"])),axis=1)
testing[:10],training[:10]

In [ ]:
len(testing),len(training)

In [ ]:
import json
json_test = dict()
for i in testing:
    json_test[i[1]] = i[0]
json.dump(json_test,open("Test.json","w"))
json_train = dict()
for i in training:
    json_train[i[1]] = i[0]
json.dump(json_train,open("Train.json","w"))
    